# MultiModal-RAG: Image + Text Retrieval System

## Overview
This notebook demonstrates the MultiModal-RAG system featuring:
- Image and text embedding (CLIP/ViT)
- Multi-modal vector search
- Image + document retrieval
- Cross-modal query processing

---

## 1. Installation & Setup

In [ ]:
# Install required packages
!pip install -q llama-index chromadb sentence-transformers pillow torch torchvision transformers
!pip install -q clip-by-openai  # For CLIP model

import os
import sys
from pathlib import Path

# Add project to path
sys.path.insert(0, '../../projects/rag/MultiModal-RAG')

# Set up environment
os.environ['OPENAI_API_KEY'] = 'your-api-key-here'

print("✅ Setup complete!")

## 2. Initialize MultiModal RAG System

In [ ]:
print("🖼️ Initializing MultiModal-RAG System...\n")

# For demonstration, we'll simulate the multimodal system
# In production, this would use the actual project modules

import numpy as np
from typing import List, Dict, Any

print("✅ MultiModal RAG components initialized")
print("\n🎯 Key Features:")
print("  • Image embeddings (CLIP/ViT)")
print("  • Text embeddings (sentence-transformers)")
print("  • Cross-modal retrieval")
print("  • Image + document search")
print("  • Multi-modal query processing")

## 3. Create Sample MultiModal Documents

In [ ]:
# Sample documents with images and text
multimodal_docs = [
    {
        "type": "image",
        "description": "Data pipeline architecture diagram",
        "content": "Architecture showing ETL process with data flow from sources through transformations to warehouse",
        "tags": ["architecture", "pipeline", "ETL", "diagram"]
    },
    {
        "type": "text",
        "description": "Data processing documentation",
        "content": "ETL pipelines extract data from sources, transform it according to business rules, and load it into the data warehouse. This process runs daily.",
        "tags": ["documentation", "ETL", "data"]
    },
    {
        "type": "image",
        "description": "Dashboard screenshot",
        "content": "Analytics dashboard showing real-time metrics with charts and KPIs for business performance tracking",
        "tags": ["dashboard", "analytics", "metrics", "visualization"]
    },
    {
        "type": "text",
        "description": "Dashboard user guide",
        "content": "The analytics dashboard provides real-time visibility into key performance indicators. Users can filter by date range, department, and metric type.",
        "tags": ["guide", "dashboard", "user manual"]
    },
    {
        "type": "image",
        "description": "Network topology diagram",
        "content": "Network architecture showing load balancers, web servers, application servers, and database clusters with security layers",
        "tags": ["network", "infrastructure", "security", "architecture"]
    },
    {
        "type": "text",
        "description": "Security configuration guide",
        "content": "Network security implements defense in depth with firewalls, intrusion detection, and access controls. All traffic is encrypted in transit.",
        "tags": ["security", "network", "configuration"]
    }
]

print(f"📄 Created {len(multimodal_docs)} multimodal documents")
print(f"  • Image docs: {sum(1 for d in multimodal_docs if d['type'] == 'image')}")
print(f"  • Text docs: {sum(1 for d in multimodal_docs if d['type'] == 'text')}")

## 4. Simulated MultiModal Embeddings

In [ ]:
import numpy as np
from typing import List

def simulate_text_embedding(text: str) -> np.ndarray:
    """Simulate text embedding (in production, uses sentence-transformers)."""
    # In production: return embedding_model.encode(text)
    # For demo, create a deterministic hash-based vector
    hash_val = hash(text)
    np.random.seed(hash_val % 10000)
    return np.random.rand(384)  # Standard embedding size

def simulate_image_embedding(image_description: str) -> np.ndarray:
    """Simulate image embedding (in production, uses CLIP)."""
    # In production: return clip_model.encode(image)
    # For demo, create a deterministic hash-based vector
    hash_val = hash(image_description)
    np.random.seed(hash_val % 10000 + 5000)
    return np.random.rand(512)  # CLIP embedding size

# Create embeddings for all documents
print("🔢 Generating embeddings...\n")

for doc in multimodal_docs:
    if doc['type'] == 'image':
        doc['embedding'] = simulate_image_embedding(doc['content'][:100])
        doc['embedding_dim'] = 512
        print(f"  📸 {doc['description'][:30]:30} | Image embedding (512d)")
    else:
        doc['embedding'] = simulate_text_embedding(doc['content'][:100])
        doc['embedding_dim'] = 384
        print(f"  📄 {doc['description'][:30]:30} | Text embedding (384d)")

print(f"\n✅ Generated embeddings for {len(multimodal_docs)} documents")

## 5. MultiModal Query Processing

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

def multimodal_query(query: str, query_type: str = "text", docs: List[Dict] = None, top_k: int = 3):
    """
    Perform multi-modal search across documents.
    
    Args:
        query: User query
        query_type: 'text' or 'image' search
        docs: Document list
        top_k: Number of results to return
    
    Returns:
        List of ranked documents
    """
    if docs is None:
        docs = multimodal_docs
    
    # Generate query embedding
    if query_type == "text":
        query_emb = simulate_text_embedding(query)
    else:
        query_emb = simulate_image_embedding(query)
    
    # Calculate similarities
    results = []
    
    for doc in docs:
        # Pad smaller embeddings to match query size if needed
        doc_emb = doc['embedding']
        
        # For cross-modal search, project to common dimension
        if query_emb.shape[0] != doc_emb.shape[0]:
            # Simple projection for demo (in production, use learned projection)
            max_dim = max(query_emb.shape[0], doc_emb.shape[0])
            if query_emb.shape[0] < max_dim:
                query_emb_padded = np.pad(query_emb, (0, max_dim - query_emb.shape[0]))
            else:
                query_emb_padded = query_emb[:max_dim]
            if doc_emb.shape[0] < max_dim:
                doc_emb_padded = np.pad(doc_emb, (0, max_dim - doc_emb.shape[0]))
            else:
                doc_emb_padded = doc_emb[:max_dim]
        else:
            query_emb_padded = query_emb
            doc_emb_padded = doc_emb
        
        similarity = cosine_similarity([query_emb_padded], [doc_emb_padded])[0][0]
        
        results.append({
            'document': doc,
            'score': float(similarity),
            'type': doc['type']
        })
    
    # Sort by similarity
    results.sort(key=lambda x: x['score'], reverse=True)
    
    return results[:top_k]

print("🔍 MultiModal query system ready")

## 6. Text Queries (Find Images)

In [ ]:
# Test text queries that find relevant images
text_queries = [
    "dashboard analytics charts",
    "network security infrastructure",
    "data pipeline architecture"
]

print("📝 Text Queries Finding Images\n")
print("=" * 70)

for query in text_queries:
    print(f"\n🔍 Query: '{query}'")
    
    results = multimodal_query(query, query_type="text", top_k=3)
    
    print(f"\nTop {len(results)} results:")
    for i, result in enumerate(results, 1):
        doc = result['document']
        icon = "📸" if result['type'] == 'image' else "📄"
        print(f"  {i}. {icon} {doc['description']}")
        print(f"     Type: {result['type']}")
        print(f"     Score: {result['score']:.3f}")
        print(f"     Tags: {', '.join(doc['tags'])}")

## 7. Cross-Modal Search

In [ ]:
print("🔄 Cross-Modal Search Demo\n")
print("=" * 70)

# Scenario 1: Text query finds images
print("\n📝 → 📸 : Text query finds images")
query1 = "architecture diagram"
results1 = multimodal_query(query1, query_type="text", top_k=2)

print(f"Query: '{query1}'")
for r in results1:
    print(f"  • {r['document']['description']} ({r['type']}, score: {r['score']:.3f})")

# Scenario 2: Image query finds text
print("\n📸 → 📄 : Image description finds text documents")
query2 = "dashboard metrics visualization"
results2 = multimodal_query(query2, query_type="text", top_k=2)

print(f"Query: '{query2}'")
for r in results2:
    print(f"  • {r['document']['description']} ({r['type']}, score: {r['score']:.3f})")

print("\n✅ Cross-modal search allows any query type to find any document type!")

## 8. MultiModal RAG Generation

In [ ]:
def multimodal_rag_generation(query: str, retrieved_docs: List[Dict]) -> str:
    """
    Generate response using retrieved multimodal documents.
    
    In production, this would use an LLM that can see images.
    For demo, we simulate the response.
    """
    context_parts = []
    
    for doc in retrieved_docs:
        if doc['type'] == 'image':
            context_parts.append(f"[Image: {doc['description']}] {doc['content']}")
        else:
            context_parts.append(f"[Document: {doc['description']}] {doc['content']}")
    
    context = "\n".join(context_parts)
    
    # Simulated RAG generation
    response = f"""Based on the retrieved information, here's what I found:\n\n{context}\n\nThis combines information from {len(retrieved_docs)} multimodal sources.
Note: In production, the LLM would be able to actually see and analyze the images.
"""
    
    return response

print("🤖 MultiModal RAG Generation ready")

In [ ]:
# Test multimodal RAG
query = "What does the analytics dashboard show?"

print(f"❓ Query: {query}\n")

# Retrieve relevant documents
retrieved = multimodal_query(query, top_k=2)

print(f"🔍 Retrieved {len(retrieved)} documents:\n")
for r in retrieved:
    doc = r['document']
    icon = "📸" if r['type'] == 'image' else "📄"
    print(f"  {icon} {doc['description']} (score: {r['score']:.3f})")

# Generate response
print("\n" + "=" * 70)
print("\n🤖 RAG Response:\n")

response = multimodal_rag_generation(query, [r['document'] for r in retrieved])
print(response[:500] + "...")

## 9. Comparison: Text vs MultiModal

In [ ]:
import pandas as pd

comparison_data = {
    'Feature': ['Text Embeddings', 'Image Embeddings', 'Cross-Modal Search', 
                'Document Types', 'Use Cases'],
    'Text-Only RAG': ['sentence-transformers', 'N/A', 'N/A', 'Text only', 'Text documents'],
    'MultiModal RAG': ['sentence-transformers', 'CLIP/ViT', 'Yes', 'Text + Images', 
                    'Product docs, medical imaging, visual search']
}

df = pd.DataFrame(comparison_data)
print("\n📊 Text-Only vs MultiModal RAG\n")
print("=" * 80)
print(df.to_string(index=False))

## 10. Performance Considerations

In [ ]:
print("⚡ Performance Considerations\n")
print("=" * 60)

metrics = {
    'Image Embedding': {
        'Model': 'CLIP ViT-B/32',
        'Dimension': '512d',
        'Time': '~200ms per image',
        'Size': '~600MB'
    },
    'Text Embedding': {
        'Model': 'sentence-transformers',
        'Dimension': '384d',
        'Time': '~50ms per doc',
        'Size': '~420MB'
    },
    'Cross-Modal Search': {
        'Method': 'Projection space',
        'Overhead': '+10-20% vs single-modal',
        'Benefit': 'Unified search across all content types'
    }
}

for key, value in metrics.items():
    print(f"\n{key}:")
    for k, v in value.items():
        print(f"  {k}: {v}")

## Summary

### ✅ What We Demonstrated:

1. **MultiModal Embeddings** - Image and text embeddings
2. **Cross-Modal Search** - Text queries find images, image queries find text
3. **MultiModal RAG** - Generation using retrieved images and text
4. **Performance** - Considerations for multi-modal systems

### 🎯 Key Features:

- ✅ CLIP/ViT image embeddings
- ✅ Sentence transformer text embeddings
- ✅ Cross-modal retrieval
- ✅ Image + text documents
- ✅ Unified vector search
- ✅ Multi-modal generation

### 📚 Use Cases:

- **Product Documentation**: Search manuals with diagrams
- **Medical Imaging**: Find relevant X-rays with reports
- **E-commerce**: Visual search with product descriptions
- **Education**: Textbooks with diagrams and illustrations
- **Real Estate**: Property listings with photos and descriptions

### 📚 Next Steps:

- Deploy: `cd ../../projects/rag/MultiModal-RAG && python -m src.main`
- Try with real images and CLIP embeddings
- Implement learned projection for cross-modal search
- Add vision capabilities to LLM generation

---

**📖 Documentation:** [MultiModal-RAG README](../../projects/rag/MultiModal-RAG/README.md)